### **Unilever Data Migration Project Test Case**

`Yoshiando M. Simangunsong`

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import time
from datetime import datetime

#### **LOAD**

This function is based on the assumption that each Excel file to be transformed has the same characteristics over time, especially for sheet names.

In [2]:
def load_excel(x):
    # Loads an excel file located at the path x and reads a specific sheet named 'Input Data - Cust Hierarchy'
        # into a pandas DataFrame called CustHierarchy.
    CustHierarchy = pd.read_excel(x, sheet_name='Input Data - Cust Hierarchy')

    # Loads the same Excel file as in the previous line and reads a different sheet named 'Input Data - FBL5N'
        # into a pandas DataFrame called FBL5N.
    FBL5N = pd.read_excel(x, sheet_name='Input Data - FBL5N')

    # Converts each datetime object in the Holiday list to a numpy datetime64 object.
    Holiday = pd.to_datetime(pd.read_excel(x, sheet_name='Input - Holiday')['Holiday'].tolist(), format='%d/%m/%y')
    Holiday = [np.datetime64(x) for x in Holiday]

    # Converts each datetime64 object in the Holiday list to a '<M8[D]' object with a specific date unit, 
        # in this case, '<M8[D]' represents the date unit of days.
    Holiday = [x.astype('<M8[D]') for x in Holiday]

    # Loads the same Excel file and reads a sheet named 'Input - Report Date', then it retrieves the first column's first value, 
        # converts it to a numpy datetime64 object, and assigns it to a variable called ReportDate.
    ReportDate = (np.datetime64(pd.read_excel(x, sheet_name='Input - Report Date').columns.values[0])).astype('<M8[D]')

    return CustHierarchy, FBL5N, Holiday, ReportDate

#### **MERGE**

This function is also based on the assumption that each Excel file to be transformed has the same characteristics over time, especially for column names.

In [3]:
def FBL5N_CustHierarchy(FBL5N, CustHierarchy):
    # Performs a left join between the FBL5N DataFrame and selected columns from CustHierarchy DataFrame based on the 'Account' and 'Customer' columns, respectively. 
        # It then drops the 'Customer' column and assigns the result to a new DataFrame called output.
    output = FBL5N.iloc[:, 0:2].merge(CustHierarchy[['Customer', 'Customer Name', 'TFL2', 'Customer Channel', 'SWL2', 'SWL3', 'Banner']], 
                                        how='left', 
                                        left_on='Account', 
                                        right_on='Customer').drop('Customer', axis=1)

    # Renames the columns in the output DataFrame to the specified names.    
    output.columns = ['Company Code', 'Account', 'Customer Name', 'Trade Format Level 2', 'Channel', 'Sales Workforce Lv2', 'Sales Workforce Lv3', 'Banner Name']

    # Concatenates the output DataFrame with columns from FBL5N DataFrame, starting from column index 2, 
        # along the columns axis and assigns the result back to output.
    output = pd.concat([output, FBL5N.iloc[:, 2:]], axis=1)

    # Fills any missing values in the columns of the output DataFrame, starting from column index 2 up to but not including column index 8, 
        # with the string value 'Not Found'.
    output.iloc[:, 2:8] = output.iloc[:, 2:8].fillna('Not Found')

    return output

#### **EXCLUDE**

This function fill the 'Exclude' column based on the following rules in priority order:

1. If the 'Reference Key 1' column contains 'CITDUMMY' or 'CITDUMMYUFS' or 'CITDUMMY UFS' or 'CITDUMMY-NKW', then mark exclude as Reference Key 1 value.
2. If the 'Amount in Local Currency' column is positive, mark as 'Positive Value'.
3. If either the 'Local Currency' or 'Document currency' column is not equal to 'IDR', mark as 'Not IDR'.
4. If the 'Document Number' column has a prefix of '58*', mark as 'Reversal document'.
5. If either the 'Reference' or 'Text' column contains 'BNPPAY' or 'bnppay', mark as 'BNPPAY'.

In [4]:
def exclude(output):
    x = []
    for i in range(len(output)):
        if ((output['Reference Key 1'][i] == 'CITDUMMYUFS') | (output['Reference Key 1'][i] == 'CITDUMMY') | (output['Reference Key 1'][i] == 'CITDUMMY-NKW') | (output['Reference Key 1'][i] == 'CITDUMMY UFS')):
            x.append(output['Reference Key 1'][i])
            pass
        elif output['Amount in local currency'][i] > 0:
            x.append('Positive Value')
            pass
        elif ((output['Local Currency'][i] != 'IDR') | (output['Document currency'][i] != 'IDR')):
            x.append('Not IDR')
            pass
        elif (round(output['Document Number'][i] / 100000000) == 58):
            x.append('Reversal document')
            pass
        elif 'bnppay' in str(output['Reference'][i]) or 'bnppay' in str(output['Text'][i]):
            x.append('BNPPAY')
            pass
        else:
            x.append(np.nan)
    output['Exclude'] = x
    return output


#### **DAYS**

This days() function will calculate the number of working days between two specified dates, a start date and an end date. The formula used in the Excel output utilizes the networkdays.intl function, which calculates the number of working days from the start date to the end date and then subtracts by 1. 

In this days() function, np.busday_count is used to calculate the number of working days, np.busday_count works without including the end date in the count. Therefore, np.busday_count will have the same value (as networkdays.intl -1) if the end date of np.busday_count does not fall on a weekend or holiday.

In [5]:
def days(output, Holiday, ReportDate):
    x = []
    for i in range(len(output)):
        if output['Clearing Document'].isnull()[i]:
            # Checks if the number of business days between the 'Document Date' column of output and ReportDate, 
                # excluding holidays specified in Holiday, is less than zero.
            if np.busday_count((np.datetime64(output['Document Date'][i])).astype('<M8[D]'), ReportDate, holidays=Holiday) < 0:
                x.append(0)
            else:
                # Appends the number of business days between the 'Document Date' column of output and ReportDate, 
                    # excluding holidays specified in Holiday, to the list x.
                x.append(np.busday_count((np.datetime64(output['Document Date'][i])).astype('<M8[D]'), ReportDate, holidays=Holiday))

        else:
            # Checks if the 'Clearing date' column of output falls on a weekend or holiday specified in Holiday.
            if np.busday_count((np.datetime64(output['Document Date'][i])).astype('<M8[D]'),(np.datetime64(output['Clearing date'][i])).astype('<M8[D]'), holidays=Holiday) <= 0:
                    x.append(0)
            else:
                # Appends the number of business days between the 'Document Date' column of output and the 'Clearing date' column of output, excluding holidays specified in Holiday, 
                    # minus one if the condition in step 13 is true, if false runs code below.             
                if (datetime.date(output['Clearing date'][i]).weekday() > 4) or ((np.datetime64(output['Clearing date'][i])).astype('<M8[D]') in Holiday):
                    x.append(np.busday_count((np.datetime64(output['Document Date'][i])).astype('<M8[D]'),(np.datetime64(output['Clearing date'][i])).astype('<M8[D]'), holidays=Holiday)-1)
                else:
                    x.append(np.busday_count((np.datetime64(output['Document Date'][i])).astype('<M8[D]'),(np.datetime64(output['Clearing date'][i])).astype('<M8[D]'), holidays=Holiday))
    output['Days'] = x
    return output

#### **AGEING_CLEARING**

This function creates and fills new columns named 'Ageing' and 'Clearing' based on the conditional statements below

In [6]:
def ageing_clearing(output):
    # Create a list of three conditions based on the values in the Days column of the output DataFrame.
    conditions = [
        output['Days'] < 2,
        output['Days'] == 2,
        output['Days'] > 2
        ]

    # Create a list of three values that correspond to the conditions in the previous step and add 
        # a new column to the output DataFrame called Ageing that is filled from the values.
    values = ['<2', '=2', '>2']
    output['Ageing'] = np.select(conditions, values, default='')

    # Add a new column to the output DataFrame called Clearing Status that is populated with the string 'Open' 
        # if the value in the Clearing Document column is null (i.e., the document has not been cleared), 
        # or 'Cleared' if the value in the Clearing Document column is not null.
    output['Clearing Status'] = ['Open' if pd.isnull(x) else 'Cleared' for x in output['Clearing Document']]

    return output

#### **GROUP BY**

This function returns two dataframes:

1. exclude_output: this dataframe shows the count of billing documents grouped by the exclude category. The index represents the exclude categories and the column Count of Billing Document represents the number of billing documents for each category. The last row of the dataframe shows the grand total for all exclude categories.  

2. channel_output: this dataframe shows the count of billing documents grouped by channel, excluding those with ageing greater than 2 and not excluded. The index represents the channels and the column Count of Billing Document represents the number of billing documents for each channel. The last row of the dataframe shows the grand total for all channels.

In [7]:
def groupby_exclude_channel(output):
    exclude_output = output[['Account']].groupby(output['Exclude'], dropna=False).count().rename(columns={'Account': 'Count of Billing Document'})
    total_count = exclude_output.sum().values[0]
    exclude_output.loc['Grand Total'] = total_count

    channel_output = output[['Account']][(output['Ageing'] != '>2') & (output['Exclude'].isnull())].groupby(output['Channel'],dropna=False).count().rename(columns={'Account': 'Count of Billing Document'})
    total_count = channel_output.sum()
    channel_output.loc['Grand Total'] = total_count
    return channel_output, exclude_output


#### **TRANSFORM**

The function is a combination of several functions above, which takes an input Excel file and an output Excel file as parameters. The function returns 3 variables, namely the output dataframe, the output dataframe grouped by exclude, and the output dataframe grouped by channel.

In [8]:
def transform(ExcelInput, ExcelOutput):
    Customer_Hierarchy, FBL5N, Holiday, ReportDate = load_excel(ExcelInput)
    output = FBL5N_CustHierarchy(FBL5N, Customer_Hierarchy)
    output = exclude(output)
    output = days(output, Holiday, ReportDate)
    output = ageing_clearing(output)
    groupby_channel, groupby_exclude = groupby_exclude_channel(output)
    output.to_excel(ExcelOutput, index=False)
    return output, groupby_channel, groupby_exclude

    - ExcelInput = Excelinput file you want to transform.
    - ExcelOutput = Destination of Exceloutput transformed file, use r' to read the file path.  
    - output = Processed dataframe.  
    - groupby_exclude = dataframe of group by exclude - count of row.  
    - groupby_channel = dataframe of group by channel - count of days.

In [9]:
# Outputs
output, groupby_channel, groupby_exclude = transform('FANGSYEN.xlsx', r'C:\Users\yoshi\OneDrive\Desktop\PRWDHK\Unilever\processed2.xlsx')

In [10]:
groupby_channel

,Count of Billing Document
Channel,
DT,536
IC,277
MT,324
Others,1
U-PRO,9
UFS,41
Grand Total,1188


In [11]:
groupby_exclude

,Count of Billing Document
Exclude,
BNPPAY,2
CITDUMMY,176
CITDUMMY UFS,2
CITDUMMY-NKW,7
CITDUMMYUFS,105
Not IDR,8
Positive Value,1278
Reversal document,2
NaN,1233


In [12]:
output

,Company Code,Account,Customer Name,Trade Format Level 2,Channel,Sales Workforce Lv2,Sales Workforce Lv3,Banner Name,Billing Document,Document Number,...,Text,Clearing Document,Clearing date,Arrears after net due date,Reference Key 1,Reference Key 2,Exclude,Days,Ageing,Clearing Status
0,2408,15061017,PE. SXGKCA QLCQAWQ EAWJQYQ YC KQIYX,GZYCAI EAQYC,MT,CYG GWIWGQAKCE,KQG QLCQGQAE,QLCQGQAE,NaN,3400015095,...,683040,3.400015e+09,2023-02-01,0,NaN,NaN,Not IDR,0,<2,Cleared
1,2408,15061063,PE. EXLXS GQJX,YWSEAWKXEWVC EAQYC,UFS,AKG CCIEAQL,QKG CQSECAI JQKQAEQ,PE. EXLXS GQJX,NaN,3400015047,...,682951,3.400015e+09,2023-02-01,0,NaN,NaN,Positive Value,0,<2,Cleared
2,2408,15061820,CLQIYYS GQAE,GZYCAI EAQYC,MT,ASG ZXECA WSLQIY,KXH KQLW,CLQIYYS GQAE,NaN,3400014851,...,NaN,3.400015e+09,2023-02-01,0,NaN,NaN,NaN,0,<2,Cleared
3,2408,15061820,CLQIYYS GQAE,GZYCAI EAQYC,MT,ASG ZXECA WSLQIY,KXH KQLW,CLQIYYS GQAE,NaN,3400015201,...,683007,3.400015e+09,2023-02-01,0,NaN,NaN,NaN,0,<2,Cleared
4,2408,15062513,CV. XSQHQ JQYQ QKQYW,YWSEAWKXEWVC EAQYC,DT,ASG ZXECA WSLQIY,QSG PZIEWQIQK,XSQHQ JQYQ,NaN,3400014901,...,683067,3.400015e+09,2023-02-01,0,NaN,NaN,NaN,0,<2,Cleared
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808,2408,15553734,JXAQOQI KZOZA CWPQIQS,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KZOZA CWPQIQS,NaN,3400016437,...,Short paid as on 04.02.20223,NaN,NaT,1,NaN,NaN,NaN,1,<2,Open
2809,2408,15553734,JXAQOQI KZOZA CWPQIQS,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KZOZA CWPQIQS,NaN,3400016437,...,NaN,3.400016e+09,2023-02-05,0,NaN,NaN,Positive Value,0,<2,Cleared
2810,2408,15612480,JXAQOQI KQAQWQIO CWKXQYQ,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KQAQWQIO CWKXQYQ,3.030839e+09,3400016445,...,NaN,NaN,NaT,3,NaN,NaN,NaN,1,<2,Open
2811,2408,15612948,JXAQOQI KZOZA CWSQAXQ,YWSEAWKXEWVC EAQYC,IC,ASG SCAX,QSG SCAX 2,JXAQOQI KZOZA CWSQAXQ,3.030868e+09,3400016441,...,NaN,NaN,NaT,-2,NaN,NaN,Positive Value,1,<2,Open


<H3><center><b>Thank you!</H3</b></center>